In [20]:
import csv
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Build up my corpus using the scraped data

Data was scraped using the "API_Caller.ipynb" notebook. You can find all the scraped data in the "collected_data" folder. 

In [21]:
# specify resume path.

resumePath = "resumes/jacky.txt"

In [22]:
# read in all the job postings

corpusDF = pd.DataFrame()

for f in os.listdir("collected_data/"):
    if (f.endswith(".csv")):
        
        print("Importing... ", f)
        
        filepath = os.path.join("collected_data", f)
        
        df = pd.read_csv(filepath, encoding = "ISO-8859-1")
        
        df["AllDescription"] = df["PositionTitle"] + " | " +  \
                                df["URI"] + " | " + df["DepartmentName"]  + " | " + \
                                df["OrganizationName"] + " | " + f  + " | " + \
                                df["JobSummary"] + " | " + \
                                df["QualificationSummary"]
        
        corpusDF = corpusDF.append(df, ignore_index = True)        
        
        print("Success")
        print("================================================================")

Importing...  aerospace_N_113_20171224133539.csv
Success
Importing...  analysis_N_500_20171223141842.csv
Success
Importing...  analyst_N_239_20171223115544.csv
Success
Importing...  attorney_N_180_20171223141257.csv
Success
Importing...  aviation_N_355_20171224133743.csv
Success
Importing...  businessintelligence_N_20_20171223115447.csv
Success
Importing...  combat_N_400_20171224133814.csv
Success
Importing...  computational_N_17_20171223115812.csv
Success
Importing...  covert_N_4_20171223141634.csv
Success
Importing...  cyber_N_76_20171223141650.csv
Success
Importing...  dataanalyst_N_96_20171223115356.csv
Success
Importing...  dataanalytics_N_19_20171223115427.csv
Success
Importing...  database_N_500_20171223120145.csv
Success
Importing...  dataengineer_N_101_20171223115828.csv
Success
Importing...  datamining_N_9_20171223120312.csv
Success
Importing...  datascience_N_346_20171223115408.csv
Success
Importing...  datascientist_N_28_20171223115414.csv
Success
Importing...  datavisualiz

Success
Importing...  sixsigma_N_6_20171223141604.csv
Success
Importing...  software_N_500_20171223115557.csv
Success
Importing...  soldier_N_162_20171224133432.csv
Success
Importing...  spatial_N_14_20171223115245.csv
Success
Importing...  SPSS_N_3_20171223120435.csv
Success
Importing...  sql_N_21_20171223120239.csv
Success
Importing...  statistical_N_277_20171223120834.csv
Success
Importing...  statistician_N_8_20171223115854.csv
Success
Importing...  statistics_N_329_20171223115916.csv
Success
Importing...  strategic_N_339_20171224133904.csv
Success
Importing...  surgeon_N_58_20171224133621.csv
Success
Importing...  systems_N_500_20171223120219.csv
Success
Importing...  tableau_N_2_20171223120948.csv
Success
Importing...  tactical_N_142_20171224133851.csv
Success
Importing...  technical_N_500_20171223115709.csv
Success
Importing...  technician_N_500_20171223141315.csv
Success
Importing...  technology_N_500_20171223120810.csv
Success
Importing...  transportation_N_500_20171224133801.

In [23]:
corpusDF.head(2)

,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,URI,AllDescription
0,2018-01-08,National Aeronautics and Space Administration,GS,The Applied Engineering and Technology Directo...,86460.0,66510.0,Permanent,Goddard Space Flight Center,Per Year,2018-01-08,GS18D0058,"Greenbelt, Maryland",2017-12-20,"Aerospace Engineer, AST, Aerospace Vehicle Des...",In addition to the Basic Education Requirement...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Aerospace Engineer, AST, Aerospace Vehicle Des..."
1,2017-12-28,Department of the Navy,GS,The selectee for this position will serve as a...,72901.0,45970.0,Permanent,"U.S. Atlantic Fleet, Commander in Chief",Per Year,2017-12-28,DE-10078062-18-HB,"Point Mugu, California",2017-12-18,AEROSPACE ENGINEER,"In order to qualify for this position, your re...",https://www.usajobs.gov:443/GetJob/ViewDetails...,AEROSPACE ENGINEER | https://www.usajobs.gov:4...


In [24]:
len(corpusDF)

18426

### There may be multiple job postings. I'll need to remove the duplicates based on the job URI. 

In [25]:
corpusDF = corpusDF.groupby("URI").first()

len(corpusDF)

5904

In [26]:
corpusDF.head(2)

,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,AllDescription
URI,,,,,,,,,,,,,,,,
https://www.usajobs.gov:443/GetJob/ViewDetails/345265800,2018-02-15,Other Agencies and Independent Organizations,GS,Thisfile serves as a repository of resumes for...,82642.00,20527.00,Multiple Appointment Types,Office of Personnel Management,Per Year,2018-02-15,US OPM Veteran File,"San Francisco, California|Denver, Colorado|Was...",2017-02-16,U.S. OPM Veteran File,As positions become available and hiring offic...,U.S. OPM Veteran File | https://www.usajobs.go...
https://www.usajobs.gov:443/GetJob/ViewDetails/357639200,2018-01-03,Department of the Air Force,WG,Air Reserve Technicians are part of the regula...,26.18,22.45,Multiple Appointment Types,"Headquarters, Air Force Reserve Command",Per Hour,2018-01-03,SEU-SR-Elec Int Sys Mech,"Elmendorf AFB, Alaska|Maxwell AFB, Alabama|Lit...",2017-01-04,Electronic Integrated System Mechanic (Militar...,A specific length of experience or training is...,Electronic Integrated System Mechanic (Militar...


In [27]:
corpus = []
corpusDF["AllDescription"].apply(lambda row: corpus.append(str(row)))
len(corpus)

5904

In [28]:
corpus[300][:1000]

"Public Health Analyst | https://www.usajobs.gov:443/GetJob/ViewDetails/468772500 | Department of Health And Human Services | Centers for Disease Control and Prevention | analyst_N_239_20171223115544.csv | The Centers for Disease Control and Prevention (CDC) is the agency Americans trust with their lives. As a global leader in public health, CDC is the nation's premier health promotion, prevention, and preparedness agency. Whether we are protecting the American people from public health threats, researching emerging diseases, or mobilizing public health programs with our domestic and international partners, we rely on our employees to make a real difference in the health and well-being of people here and around the world. Who May Apply: Open to US citizens. This vacancy is also being announced concurrently with vacancy announcementHHS-CDC-OM-17-1949748 under merit promotion procedures. Please review that announcement to see if you are eligible for consideration under merit promotion pr

## Prepend my resume onto the corpus list

In [29]:
file = open(resumePath, "r")
resume = file.read()

# preprend to corpus
corpus.insert(0, resume)

In [30]:
# make sure that the first entry is the resume

corpus[0][:1000]

'Relevant Coursework: Machine Learning, High Performance Computing, Predictive Analytics, Advanced Statistics, Natural Language Processing University of Michigan                                        Ann Arbor, MI Bachelor of Science: Industrial and Operations EngineeringDate of Graduation: April 2011Relevant Coursework: Software Development, Optimization Methods, Linear Statistical Models, Markov Process, Operations Modeling, Statistics ASQ Certified Six Sigma Black BeltLicense Number: 12771 WORK EXPERIENCE: Accenture                                                            Austin, TX  Senior Software Development Engineer                                                                       May 2014 – Present  ?Developed and maintained healthcare web applications (.NET MVC C#, HTML, Javascript, CSS, SQL Server) for the Texas Health and Human Services Commission (HHSC) to process Medicaid claims, patient forms, and physician inventory status. Collaborated with clients to translate b

## Build TF-IDF vectorizer and matrix

In [31]:
tf = TfidfVectorizer(analyzer = "word", 
                        ngram_range = (1, 3),
                        min_df = 0, 
                        stop_words = "english")

In [32]:
matrix = tf.fit_transform(corpus)

## Find similar jobs using cosine similarity

In [33]:
def find_similar(matrix, index, top_n = 10):
    cosine_similarities = linear_kernel(matrix[index: index + 1], matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

## Let's test this out

In [34]:
k = 0

print("RESUME:\n")
print(corpus[k][:150000])

print("\n============================================================================\n")
print("MATCHED JOBS: \n")
for index, score in find_similar(matrix, k):
    print("\n")
    print(score, corpus[index][:3000])
    print("\n...next job...")

RESUME:

Relevant Coursework: Machine Learning, High Performance Computing, Predictive Analytics, Advanced Statistics, Natural Language Processing University of Michigan                                        Ann Arbor, MI Bachelor of Science: Industrial and Operations EngineeringDate of Graduation: April 2011Relevant Coursework: Software Development, Optimization Methods, Linear Statistical Models, Markov Process, Operations Modeling, Statistics ASQ Certified Six Sigma Black BeltLicense Number: 12771 WORK EXPERIENCE: Accenture                                                            Austin, TX  Senior Software Development Engineer                                                                       May 2014 – Present  ?Developed and maintained healthcare web applications (.NET MVC C#, HTML, Javascript, CSS, SQL Server) for the Texas Health and Human Services Commission (HHSC) to process Medicaid claims, patient forms, and physician inventory status. Collaborated with clients to tra